In [1]:
from vivarium import Artifact
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from db_queries import get_ids, get_outputs
import scipy.stats

!whoami
!date

alibow
Thu Jun 18 14:22:42 PDT 2020


In [2]:
output_dirs = ['/share/costeffectiveness/results/vivarium_conic_lsff/00_bugfix/v7.0_prelim_nigeria/count_data/']

locations = ['Nigeria']

In [3]:
pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')
pt.head()

,year,age_group,sex,folic_acid_fortification_group,vitamin_a_fortification_group,measure,input_draw,scenario,value
0,2020,1_to_4,female,covered,covered,person_time,357,baseline,0.0
1,2020,1_to_4,female,covered,covered,person_time,357,folic_acid_fortification_scale_up,0.0
2,2020,1_to_4,female,covered,covered,person_time,357,iron_fortification_scale_up,0.0
3,2020,1_to_4,female,covered,covered,person_time,357,vitamin_a_fortification_scale_up,0.0
4,2020,1_to_4,female,covered,covered,person_time,602,baseline,0.0


In [4]:
np.unique(pt['folic_acid_fortification_group'])

array(['covered', 'uncovered', 'unknown'], dtype=object)

In [5]:
np.unique(pt['vitamin_a_fortification_group'])

array(['covered', 'effectively_covered', 'uncovered'], dtype=object)

# Vitamin A Fortification Coverage

In [12]:
# COMPUTE COVERAGE PROPORTION BY LOCATION, SCENARIO, YEAR

data = pt = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario.isin(['baseline','vitamin_a_fortification_scale_up'])] # look only at baseline and vitamin a scenario
covered = data.loc[data.vitamin_a_fortification_group != 'uncovered'] # look at effectively covered and covered groups
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

overall = data.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / overall
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                         year                                        
baseline                         2020    2.0  0.068356  0.004164  0.065411   
                                 2021    2.0  0.067676  0.003829  0.064969   
                                 2022    2.0  0.066819  0.003009  0.064692   
                                 2023    2.0  0.068063  0.004215  0.065082   
                                 2024    2.0  0.069313  0.003551  0.066802   
                                 2025    2.0  0.068800  0.003735  0.066159   
vitamin_a_fortification_scale_up 2020    2.0  0.338046  0.002540  0.336250   
                                 2021    2.0  0.397572  0.003317  0.395227   
                                 2022    2.0  0.443215  0.003133  0.441000   
                                 2023    2.0  0.471663  0.007463  0.466386   
                                 2024    2.0  0.490103  0.008168  0.484328   
                                 2025    2.0  0.495194  0.005406  0.491372   

                                            25%       50%       75%       max  
scenario                         year                                          
baseline                         2020  0.066884  0.068356  0.069828  0.071301  
                                 2021  0.066323  0.067676  0.069030  0.070384  
                                 2022  0.065756  0.066819  0.067883  0.068947  
                                 2023  0.066572  0.068063  0.069553  0.071043  
                                 2024  0.068057  0.069313  0.070568  0.071824  
                                 2025  0.067479  0.068800  0.070120  0.071441  
vitamin_a_fortification_scale_up 2020  0.337148  0.338046  0.338944  0.339842  
                                 2021  0.396399  0.397572  0.398745  0.399917  
                                 2022  0.442107  0.443215  0.444323  0.445430  
                                 2023  0.469025  0.471663  0.474302  0.476941  
                                 2024  0.487216  0.490103  0.492991  0.495879  
                                 2025  0.493283  0.495194  0.497105  0.499016

## Vitamin A coverage conclusion


- coverage is starting in 2020 still, not 2021, although the coverage value that intervention scenario doesn't make sense to me
- based on the coverage algorithm, the 2021 coverage in the intervention scenario should be 89%, which it is.
- not sure why 2020 intervention scenario coverage in 2020 is 80%... this is outside of the confidence interval for parameter b in the coverage algorithm

# Folic Acid Fortification Coverage

In [11]:
# COMPUTE COVERAGE PROPORTION BY SCENARIO, YEAR - Folic acid

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario.isin(['baseline','folic_acid_fortification_scale_up'])]
covered = data.loc[data.folic_acid_fortification_group == 'covered']
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

known_coverage = data.loc[data.folic_acid_fortification_group != 'unknown']
known_coverage = known_coverage.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / known_coverage
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

count      mean       std       min  \
scenario                          year                                        
baseline                          2020    2.0  0.074528  0.009572  0.067760   
                                  2021    2.0  0.080997  0.007284  0.075846   
                                  2022    2.0  0.081662  0.005477  0.077789   
                                  2023    2.0  0.081131  0.006090  0.076825   
                                  2024    2.0  0.081809  0.005906  0.077633   
                                  2025    2.0  0.081891  0.005114  0.078275   
folic_acid_fortification_scale_up 2020    2.0  0.074528  0.009572  0.067760   
                                  2021    2.0  0.080997  0.007284  0.075846   
                                  2022    2.0  0.092741  0.004808  0.089342   
                                  2023    2.0  0.139318  0.003508  0.136837   
                                  2024    2.0  0.167668  0.005592  0.163714   
                                  2025    2.0  0.181418  0.005731  0.177366   

                                             25%       50%       75%       max  
scenario                          year                                          
baseline                          2020  0.071144  0.074528  0.077913  0.081297  
                                  2021  0.078421  0.080997  0.083572  0.086147  
                                  2022  0.079725  0.081662  0.083598  0.085535  
                                  2023  0.078978  0.081131  0.083285  0.085438  
                                  2024  0.079721  0.081809  0.083897  0.085985  
                                  2025  0.080083  0.081891  0.083699  0.085507  
folic_acid_fortification_scale_up 2020  0.071144  0.074528  0.077913  0.081297  
                                  2021  0.078421  0.080997  0.083572  0.086147  
                                  2022  0.091042  0.092741  0.094441  0.096141  
                                  2023  0.138077  0.139318  0.140558  0.141798  
                                  2024  0.165691  0.167668  0.169645  0.171622  
                                  2025  0.179392  0.181418  0.183445  0.185471

## Folic Acid Fortification Coverage Conclusion
- Looks as expected based on coverage algorithm, although differences are slightly hard to discern due to very close values for parameters a and b in algorithm


# Iron Fotification Intervention Coverage

In [14]:
# COMPUTE COVERAGE PROPORTION BY SCENARIO, YEAR - Folic acid

data = pd.read_hdf(output_dirs[0] + 'person_time.hdf')

data = data.loc[data.scenario.isin(['baseline','iron_fortification_scale_up'])]
covered = data.loc[data.folic_acid_fortification_group == 'covered']
covered = covered.groupby(['scenario','year','input_draw']).sum() # sum over age group, sex

known_coverage = data.loc[data.iron_fortification_group != 'unknown']
known_coverage = known_coverage.groupby(['scenario','year','input_draw']).sum()

coverage_proportion = covered / known_coverage
coverage_proportion = (coverage_proportion.reset_index()
                       .groupby(['scenario','year']).describe())
coverage_proportion['value']

AttributeError: 'DataFrame' object has no attribute 'iron_fortification_group'

In [15]:
# we need an "iron_fortification_coverage_group" column